# 2021_1_11

Quick notebook to load matlab struct from tyler's code & see if it imports into python: https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries

In [17]:
import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

In [21]:
def load_mat(filename):
    """
    This function should be called instead of direct scipy.io.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects. From https://stackoverflow.com/questions
    /48970785/complex-matlab-struct-mat-file-read-by-python
    """

    def _check_vars(d):
        """
        Checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        """
        for key in d:
            if isinstance(d[key], matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
            elif isinstance(d[key], np.ndarray):
                d[key] = _toarray(d[key])
        return d

    def _todict(matobj):
        """
        A recursive function which constructs from matobjects nested dictionaries
        """
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _toarray(elem)
            else:
                d[strg] = elem
        return d

    def _toarray(ndarray):
        """
        A recursive function which constructs ndarray from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        """
        if ndarray.dtype != 'float64':
            elem_list = []
            for sub_elem in ndarray:
                if isinstance(sub_elem, matlab.mio5_params.mat_struct):
                    elem_list.append(_todict(sub_elem))
                elif isinstance(sub_elem, np.ndarray):
                    elem_list.append(_toarray(sub_elem))
                else:
                    elem_list.append(sub_elem)
            return np.array(elem_list)
        else:
            return ndarray

    data = loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_vars(data)

In [18]:
test = loadmat('ttl_match.mat')

In [19]:
test["res"]["sessiondate"]

'2019-07-24'

In [20]:
test

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Jan 11 11:48:15 2021',
 '__version__': '1.0',
 '__globals__': [],
 'sess': 'data_sdb_20190724_193007_fromSD',
 'res': {'goodval': 0.0004963520118508313,
  'gooddur': 59.3619668,
  'spk2fsm_rt': array([ 0.99995139, 35.3585892 ]),
  'spk2fsm_fn': MatlabFunction(<scipy.io.matlab.mio5_params.mat_struct object at 0x0000020EA24E28E0>,
                 dtype=object),
  'goodpath': 'Y:\\RATTER\\SoloData\\Data\\Emily\\W122\\data_@PWM_Emily_W122_190724a.mat',
  'behfile': 'data_@PWM_Emily_W122_190724a.mat',
  'ratname': 'W122',
  'sessid': 716087,
  'sessiondate': '2019-07-24',
  'date_str': '190724',
  'save_path': 'W:\\jbreda\\ephys\\W122\\data_sdb_20190724_193007_fromSD.mda\\ttl_match.mat'},
 '__function_workspace__': array([[ 0,  1, 73, ...,  0,  0,  0]], dtype=uint8)}

In [2]:
import numpy as np
np_test1 = np.load('Y:/jbreda/ephys/post_sort_analysis/bin_mask_info/data_sdc_20190902_145404_fromSD_firstbundle_mask_info')
np_test2 = np.load('Y:/jbreda/ephys/post_sort_analysis/bin_mask_info/data_sdc_20190902_145404_fromSD_secondbundle_mask_info')
np_test3 = np.load('Y:/jbreda/ephys/post_sort_analysis/bin_mask_info/data_sdc_20190902_145404_fromSD_thirdbundle_mask_info')
np_test4 = np.load('Y:/jbreda/ephys/post_sort_analysis/bin_mask_info/data_sdc_20190902_145404_fromSD_fourthbundle_mask_info')

In [3]:
np.array_equal(np_test1,np_test2)

False

-7422007

(array([0, 1], dtype=int64),)